In [1]:
import bt
import plotly.express as px
from datetime import datetime
from dateutil.relativedelta import relativedelta

from data.get_data import get_prices_df, get_pdf_df

from core.get_backtest import get_base_backtest, get_eql_backtest, get_mkw_backtest, get_bdd_mkw_backtest
from core.get_weigh import get_cap_weigh, get_bdd_cap_weigh


class CONFIG:
    ticker = "AIQ"
    
    upper_bound=0.03
    
    start_date = datetime.now() - relativedelta(years=1)
    
    template="simple_white"
    primary_color="#F58220"
    secondary_color="#CDCECB"
    
    
def get_miraeasset_theme_figure(data, title):
    fig = px.line(data, 
              template=CONFIG.template,
              title=title,
              labels={
                  "value": "YTD",
                  "index": "date",
                  "variable": "ETFs"
              })

    fig['data'][0]['line']['color']=CONFIG.primary_color
    fig['data'][1]['line']['color']=CONFIG.secondary_color
    
    return fig



pdf_df = get_pdf_df(etf_tkr=CONFIG.ticker)

tkrs = pdf_df.child_stk_tkr.to_list()

etf_price = bt.get(CONFIG.ticker, start=CONFIG.start_date)
etf_price = etf_price.dropna()

child_prices = get_prices_df(tickers=tkrs, start_date=CONFIG.start_date)

[*********************100%***********************]  1 of 1 completed


### 01 동일가중
- 리밸런싱: 3개월

In [18]:
base_backtest = get_base_backtest(name=f"{CONFIG.ticker}", etf_price=etf_price)
eql_backtest = get_eql_backtest(name=f"{CONFIG.ticker} 동일가중", child_prices=child_prices)

ret = bt.run(eql_backtest, base_backtest)

TITLE=f"{CONFIG.ticker} vs {CONFIG.ticker} 동일가중"

fig = get_miraeasset_theme_figure(data=ret._get_series(freq=None).rebase(),
                                  title=TITLE)

fig.show()

In [24]:
ret.get_weights().iloc[:5]

,AIQ 동일가중,AIQ 동일가중>aapl,AIQ 동일가중>acn,AIQ 동일가중>adbe,AIQ 동일가중>amba,AIQ 동일가중>amzn,AIQ 동일가중>bah,AIQ 동일가중>bidu,AIQ 동일가중>bz,AIQ 동일가중>cccs,...,AIQ 동일가중>tsla,AIQ 동일가중>ttd,AIQ 동일가중>twlo,AIQ 동일가중>uber,AIQ 동일가중>vrnt,AIQ 동일가중>vsat,AIQ 동일가중>wday,AIQ 동일가중>wix,AIQ 동일가중>zbra,AIQ 동일가중>zs
2022-12-14,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-12-15,1.0,0.016854,0.016672,0.016764,0.016879,0.016894,0.016869,0.016941,0.016931,0.016947,...,0.016871,0.016934,0.016914,0.016925,0.016916,0.016933,0.016840,0.016912,0.016820,0.016872
2022-12-16,1.0,0.016815,0.015881,0.017481,0.016475,0.016990,0.017029,0.016985,0.016965,0.017158,...,0.016275,0.016915,0.017078,0.016959,0.017046,0.016818,0.017031,0.017035,0.016886,0.016651
2022-12-19,1.0,0.016861,0.015626,0.017297,0.016188,0.016733,0.017303,0.017228,0.017205,0.017364,...,0.016543,0.016481,0.016817,0.016602,0.017379,0.016667,0.016840,0.016896,0.016845,0.016678
2022-12-20,1.0,0.016738,0.015915,0.017674,0.016542,0.016672,0.017189,0.016505,0.017259,0.017365,...,0.015108,0.017281,0.016937,0.016496,0.017087,0.017754,0.016928,0.017071,0.016967,0.016752


### 02 시가총액가중(유동시가총액)
- 리밸런싱: 3개월

In [3]:
quartely_child_cap_weigh = get_cap_weigh(child_prices=child_prices, pdf_df=pdf_df)

base_backtest = get_base_backtest(name=f"{CONFIG.ticker}", etf_price=etf_price)
mkw_backtest = get_mkw_backtest(name=f"{CONFIG.ticker} 시총가중",
                                child_prices=child_prices,
                                weigh=quartely_child_cap_weigh)

ret = bt.run(mkw_backtest, base_backtest)

TITLE=f"{CONFIG.ticker} vs {CONFIG.ticker} 시총가중"

fig = get_miraeasset_theme_figure(data=ret._get_series(freq=None).loc[quartely_child_cap_weigh.index[0]:].rebase(),
                                  title=TITLE)

fig.show()

### 03 최대제한 시가총액가중(유동비율)
- 리밸런싱 3개월

In [4]:
quartely_child_cap_upper_bound_weigh = get_bdd_cap_weigh(child_prices=child_prices,
                                                         pdf_df=pdf_df,
                                                         upper_bound=CONFIG.upper_bound)

base_backtest = get_base_backtest(name=f"{CONFIG.ticker}", etf_price=etf_price)
bdd_mkw_backtest = get_bdd_mkw_backtest(name=f"{CONFIG.ticker} 최대제한 시총가중",
                                child_prices=child_prices,
                                weigh=quartely_child_cap_upper_bound_weigh)

ret = bt.run(bdd_mkw_backtest, base_backtest)

TITLE=f"{CONFIG.ticker} vs {CONFIG.ticker} 최대제한 시총가중"

fig = get_miraeasset_theme_figure(data=ret._get_series(freq=None).loc[quartely_child_cap_upper_bound_weigh.index[0]:].rebase(),
                                  title=TITLE)

fig.show()

### 04 (2번 통신 response) API 통신을 위한 JSON 형식 변환

In [5]:
import pandas as pd


weighs = pd.DataFrame()

weighs['mkw_ratio'] = quartely_child_cap_weigh.iloc[-1].round(4)
weighs['umkw_ratio'] = quartely_child_cap_upper_bound_weigh.iloc[-1].round(4)
weighs['eql_ratio'] = 1/weighs.shape[0]
weighs['eql_ratio'] = weighs['eql_ratio'].round(4)

weighs = weighs.reset_index().rename(columns={"index": "child_stk_tkr"})

weighs['child_stk_tkr'] = weighs['child_stk_tkr'].str.upper()

pdf = get_pdf_df(etf_tkr=CONFIG.ticker)

weighs = weighs.merge(pdf[['child_stk_tkr', 'child_stk_name']], on='child_stk_tkr', how='left')

# weighs.T.to_dict()

### 05 (3번 통신 response) API 통신을 위한 누적수익률 및 리밸런싱 내역 추출
- 누적수익률 추출(myEtfYtd)

In [35]:
base_backtest = get_base_backtest(name=f"{CONFIG.ticker}", etf_price=etf_price)

user_weigh = quartely_child_cap_upper_bound_weigh.copy()
user_etf_name = f"{CONFIG.ticker} 김밍두기요오"
user_backtest = get_bdd_mkw_backtest(name=user_etf_name,
                                    child_prices=child_prices,
                                    weigh=user_weigh)

ret = bt.run(user_backtest, base_backtest)

cumulative_ret = ret._get_series(freq=None).loc[quartely_child_cap_upper_bound_weigh.index[0]:].rebase()
 
cumulative_ret = cumulative_ret[user_etf_name].reset_index().rename(columns={"index": "date", user_etf_name: "ytd"})
cumulative_ret["date"] = cumulative_ret["date"].apply(lambda x: str(x).split(' ')[0])

cumulative_ret["ytd"].to_list()[:10]

[100.0,
 99.79650505199432,
 101.36299960985184,
 98.77790426511766,
 100.83690058202745,
 102.34082855157854,
 103.17318270130158,
 105.13794817581177,
 105.78450499525071,
 106.6594055490494]

- 리밸런싱 내역 추출

In [58]:
rebalance = ret.get_weights().loc[quartely_child_cap_upper_bound_weigh.index[0]:].drop(user_etf_name, axis=1)
rebalance.columns = rebalance.columns.str.replace(user_etf_name+">", "")

for col in rebalance.columns:
    print(rebalance[col].to_list()[:10])
    break

[0.029896789154052736, 0.02974402759348616, 0.02991865926097077, 0.028555617548089364, 0.027988573372549813, 0.02848239575989061, 0.02842277319574017, 0.028839311049995913, 0.030080984689942785, 0.029951911848260265]


- 낙폭(Drawdown) 추출

In [34]:
drawdown_series = ret._get_series(freq=None).loc[quartely_child_cap_upper_bound_weigh.index[0]:].to_drawdown_series()
drawdown_series[user_etf_name].to_list()[:10]

[0.0,
 -0.0020349494800568912,
 0.0,
 -0.02550334298199808,
 -0.005190247228765488,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]